In [ ]:
import obspy
from obspy.signal.filter import bandpass
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox, ttk

from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure

#   FUNGSI PEMBACA DATA
def read_seismic_data(mseed_file, xml_file):
    try:
        st = obspy.read(mseed_file)
        inv = obspy.read_inventory(xml_file)  # StationXML
        return st, inv
    except Exception as e:
        messagebox.showerror("Error", f"Error membaca file: {str(e)}")
        return None, None

# Ambil informasi STASIUN dari StationXML
def extract_station_info(xml_file):
    try:
        inv = obspy.read_inventory(xml_file)
        net = inv.networks[0]
        sta = net.stations[0]

        latitude = sta.latitude
        longitude = sta.longitude
        elevation = sta.elevation
        station_code = sta.code

        return latitude, longitude, elevation, station_code

    except Exception as e:
        messagebox.showerror("Error", f"Gagal membaca info stasiun dari XML: {str(e)}")
        return None, None, None, None

#   FILTER & SIMULASI
def filter_data(stream, freqmin, freqmax):
    st_f = stream.copy()
    for tr in st_f:
        tr.data = bandpass(tr.data, freqmin=freqmin, freqmax=freqmax,
                           df=tr.stats.sampling_rate, corners=4, zerophase=True)
    return st_f

def apply_wood_anderson(stream, inv):
    wa_stream = stream.copy()
    
    paz_wa = {
        "poles": [-5.49779 - 5.60886j, -5.49779 + 5.60886j],
        "zeros": [0j, 0j],
        "gain": 1,
        "sensitivity": 1
    }

    for tr in wa_stream:
        tr.simulate(paz_remove=None, paz_simulate=paz_wa, water_level=60)

    return wa_stream

def calculate_amplitude_max(stream):
    result = {}
    for tr in stream:
        result[tr.id] = np.max(np.abs(tr.data))
    return result

#   PERHITUNGAN MAGNITUDO
def calculate_magnitude_local(max_amp, dist_km):
    return np.log10(max_amp) + 3 * np.log10(8 * dist_km) - 2.92

def calculate_magnitude_surface(max_amp, dist_km):
    return np.log10(max_amp) + 1.66 * np.log10(dist_km) + 3.3

def calculate_magnitude_body(max_amp, dist_km):
    return np.log10(max_amp) + 0.75 * np.log10(dist_km) + 2.5



#   PLOT SEISMOGRAM (SCROLLABLE)

def plot_seismogram():

    mseed_file = entry_mseed.get()
    if not mseed_file:
        messagebox.showwarning("Error", "Harap pilih file MSEED terlebih dahulu.")
        return

    try:
        st = obspy.read(mseed_file)
    except Exception as e:
        messagebox.showerror("Error", f"Gagal membaca MSEED: {str(e)}")
        return

    # Ambil komponen
    comps = {"BHZ": None, "BHN": None, "BHE": None}

    for tr in st:
        ch = tr.stats.channel
        if ch.endswith("Z"):
            comps["BHZ"] = tr
        elif ch.endswith("N"):
            comps["BHN"] = tr
        elif ch.endswith("E"):
            comps["BHE"] = tr

    #   WINDOW BARU UNTUK GRAFIK

    win = tk.Toplevel(root)
    win.title("Seismogram BHZ / BHN / BHE")
    win.geometry("950x750")

    # Frame utama + canvas + scrollbar
    outer_frame = tk.Frame(win)
    outer_frame.pack(fill="both", expand=True)

    canvas = tk.Canvas(outer_frame)
    canvas.pack(side="left", fill="both", expand=True)

    scrollbar = tk.Scrollbar(outer_frame, orient="vertical", command=canvas.yview)
    scrollbar.pack(side="right", fill="y")

    canvas.configure(yscrollcommand=scrollbar.set)
    canvas.bind('<Configure>', lambda e: canvas.configure(scrollregion=canvas.bbox("all")))

    # Frame internal tempat grafik disimpan
    inner_frame = tk.Frame(canvas)
    canvas.create_window((0, 0), window=inner_frame, anchor="nw")

    #   BUAT FIGURE (TINGGI BESAR)
    fig = Figure(figsize=(9, 9), dpi=100)

    # Tambahkan subplot
    axes_order = ["BHZ", "BHN", "BHE"]
    for i, comp in enumerate(axes_order, 1):
        tr = comps[comp]
        if tr is not None:
            ax = fig.add_subplot(3, 1, i)
            t = np.linspace(0, tr.stats.npts / tr.stats.sampling_rate, tr.stats.npts)
            ax.plot(t, tr.data)
            ax.set_title(f"Seismogram {comp}", fontsize=12)
            ax.set_xlabel("Waktu (detik)")
            ax.set_ylabel("Amplitudo")

    fig.tight_layout()

    canvas_plot = FigureCanvasTkAgg(fig, master=inner_frame)
    canvas_plot.draw()
    canvas_plot.get_tk_widget().pack(pady=10)

def browse_mseed():
    file_path = filedialog.askopenfilename(filetypes=[("MSEED Files", "*.mseed")])
    entry_mseed.delete(0, tk.END)
    entry_mseed.insert(0, file_path)

def browse_xml():
    file_path = filedialog.askopenfilename(filetypes=[("StationXML", "*.xml")])
    entry_xml.delete(0, tk.END)
    entry_xml.insert(0, file_path)

def calculate_magnitudes():
    mseed_file = entry_mseed.get()
    xml_file = entry_xml.get()

    if not mseed_file or not xml_file:
        messagebox.showwarning("Input Error", "Harap pilih file mseed dan xml.")
        return

    try:
        epicentral_distance_km = float(entry_distance.get())
    except:
        messagebox.showwarning("Input Error", "Jarak epicenter harus angka.")
        return

    st, inv = read_seismic_data(mseed_file, xml_file)
    if st is None:
        return

    # Ambil info stasiun
    lat, lon, elevation, code = extract_station_info(xml_file)

    for item in table_info.get_children():
        table_info.delete(item)

    table_info.insert("", "end", values=(
        f"{lat:.4f}" if lat else "-",
        f"{lon:.4f}" if lon else "-",
        f"{elevation:.2f}" if elevation else "-",
        code if code else "-"
    ))

    st_filtered = filter_data(st, 0.01, 20)
    st_wa = apply_wood_anderson(st_filtered, inv)

    amp_max = calculate_amplitude_max(st_wa)

    text_result.delete(1.0, tk.END)

    for tr_id, amp in amp_max.items():
        if var_magnitude.get() == "local":
            mg = calculate_magnitude_local(amp, epicentral_distance_km)
            text_result.insert(tk.END, f"ML untuk {tr_id}: {mg:.2f}\n")

        elif var_magnitude.get() == "surface":
            mg = calculate_magnitude_surface(amp, epicentral_distance_km)
            text_result.insert(tk.END, f"Ms untuk {tr_id}: {mg:.2f}\n")

        elif var_magnitude.get() == "body":
            mg = calculate_magnitude_body(amp, epicentral_distance_km)
            text_result.insert(tk.END, f"mb untuk {tr_id}: {mg:.2f}\n")

#   GUI UTAMA
root = tk.Tk()
root.title("Magnitudo + Seismogram Viewer (StationXML)")
root.configure(bg="#d6e6f2")

font_label = ("Times New Roman", 12)
font_button = ("Times New Roman", 10, "bold")

# Input mseed
tk.Label(root, text="Input file (.mseed):", font=font_label, bg="#d6e6f2").grid(row=0, column=0, sticky="e")
entry_mseed = tk.Entry(root, width=50, font=font_label)
entry_mseed.grid(row=0, column=1, padx=10, pady=5)
tk.Button(root, text="Browse", command=browse_mseed, bg="#6d597a", fg="white",
          font=font_button).grid(row=0, column=2)

# Input xml (StationXML)
tk.Label(root, text="Input file StationXML (.xml):", font=font_label, bg="#d6e6f2").grid(row=1, column=0, sticky="e")
entry_xml = tk.Entry(root, width=50, font=font_label)
entry_xml.grid(row=1, column=1, padx=10, pady=5)
tk.Button(root, text="Browse", command=browse_xml, bg="#6d597a", fg="white",
          font=font_button).grid(row=1, column=2)

# Pilih jenis magnitudo
tk.Label(root, text="Pilih jenis magnitudo:", font=font_label, bg="#d6e6f2").grid(row=2, column=0, sticky="e")

frame_radio = tk.Frame(root, bg="#d6e6f2")
frame_radio.grid(row=2, column=1, columnspan=3, sticky="w")

var_magnitude = tk.StringVar(value="local")

tk.Radiobutton(frame_radio, text="ML", variable=var_magnitude, value="local",
               bg="#d6e6f2", font=font_label).pack(side="left", padx=20)

tk.Radiobutton(frame_radio, text="MS", variable=var_magnitude, value="surface",
               bg="#d6e6f2", font=font_label).pack(side="left", padx=20)

tk.Radiobutton(frame_radio, text="MB", variable=var_magnitude, value="body",
               bg="#d6e6f2", font=font_label).pack(side="left", padx=20)


# Jarak
tk.Label(root, text="Jarak epicenter (km):", font=font_label, bg="#d6e6f2").grid(row=3, column=0, sticky="e")
entry_distance = tk.Entry(root, width=20, font=font_label)
entry_distance.grid(row=3, column=1)
entry_distance.insert(0, "50.0")

# Tombol hitung
tk.Button(root, text="Hitung Magnitudo", command=calculate_magnitudes,
          bg="#6d597a", fg="white", font=font_button).grid(row=4, column=1, pady=10)

# Hasil magnitudo
text_result = tk.Text(root, height=6, width=60, font=font_label, bg="#f2d7ee")
text_result.grid(row=5, column=0, columnspan=3, padx=10, pady=10)

# Tabel STATION info
tk.Label(root, text="Informasi STASIUN dari XML:", font=font_label, bg="#d6e6f2").grid(row=1, column=0, sticky="w")

columns = ("Latitude", "Longitude", "Elevation", "Station_code")
table_info = ttk.Treeview(root, columns=columns, show="headings", height=3)

table_info.heading("Latitude", text="Latitude")
table_info.heading("Longitude", text="Longitude")
table_info.heading("Elevation", text="Elevasi (m)")
table_info.heading("Station_code", text="Kode Stasiun")

table_info.grid(row=7, column=0, columnspan=3, padx=10, pady=10)

# Tombol Plot
tk.Button(root, text="Plot Seismogram BHZ/BHN/BHE", command=plot_seismogram,
          bg="#5f6caf", fg="white", font=font_button).grid(row=8, column=1, pady=10)

root.mainloop()